# EDA

Preprocess and analyze datasets from https://data.mos.ru

## Data preparation

In [ ]:
from os import chdir, path, getcwd
from pathlib import Path

chdir(Path(path.realpath("__file__")).parents[0].parents[0])
getcwd()

In [ ]:
import pandas as pd

# Театры https://data.mos.ru/opendata/531
theatres = pd.read_json(
    "https://data.mos.ru/odata/export/catalog?idFile=270312",
    compression="zip",
    encoding="windows-1251",
)

# Объекты культурного наследия федерального значения https://data.mos.ru/opendata/62302
cultural = pd.read_json(
    "https://data.mos.ru/odata/export/catalog?idFile=262466",
    compression="zip",
    encoding="windows-1251",
)

In [ ]:
from travel_agent.data.theatres import processed_theatres
from travel_agent.data.cultural import processed_cultural

datasets = {
    "theatres": processed_theatres(theatres),
    "cultural": processed_cultural(cultural),
}

for name, dataset in datasets.items():
    dataset["type"] = name

merged = pd.concat([datasets[dataset] for dataset in datasets])
merged.to_csv("data/merged_mosru.csv", index=False)
merged.head()